In [3]:
import json
import networkx as nx
import math
import os

# Cargar datos del archivo JSON de resumen
file_path = r'./../json_files/filtered_summary.json'
with open(file_path, 'r') as file:
    data = json.load(file)

# Cargar datos de los arcos desde el archivo JSON
arcs_file_path = r'./../json_files/character_to_arcs.json'
with open(arcs_file_path, 'r') as file:
    arcs_data = json.load(file)

# Lista de arcos ordenados cronológicamente
sorted_arcs = [
    "Romance Dawn Arc", "Orange Town Arc", "Syrup Village Arc", "Baratie Arc", "Arlong Park Arc", "Loguetown Arc",
    "Reverse Mountain Arc", "Whisky Peak Arc", "Little Garden Arc", "Drum Island Arc", "Arabasta Arc", "Jaya Arc",
    "Skypiea Arc", "Long Ring Long Land Arc", "Water 7 Arc", "Enies Lobby Arc", "Post-Enies Lobby Arc",
    "Thriller Bark Arc", "Sabaody Archipelago Arc", "Amazon Lily Arc", "Impel Down Arc", "Marineford Arc",
    "Post-War Arc", "Return to Sabaody Arc", "Fish-Man Island Arc", "Punk Hazard Arc", "Dressrosa Arc", "Zou Arc",
    "Whole Cake Island Arc", "Levely Arc", "Wano Country Arc", "Egghead Arc"
]

# Dividir los arcos en 5 segmentos
num_steps = 5
segment_size = len(sorted_arcs) // num_steps
remainder = len(sorted_arcs) % num_steps

# Crear los segmentos, asegurándose de que todos los arcos sean cubiertos
arc_segments = []
start = 0
for i in range(num_steps):
    # Asignar el tamaño base más el "extra" si es el segmento con el residuo
    end = start + segment_size + (1 if i < remainder else 0)
    arc_segments.append(sorted_arcs[start:end])
    start = end

# Generar una red para cada segmento de arcos
for i, segment in enumerate(arc_segments):
    # Crear un nuevo grafo
    G_segment = nx.Graph()

    # Iterar sobre los datos para añadir nodos y relaciones basados en los arcos del segmento
    for person, details in data.items():
        word_count = details.get("word_count", 0)
        mutual_links = details.get("mention_intersection", [])
        character_arcs = arcs_data.get(person, [])
        
        # Verificar si el personaje pertenece a alguno de los arcos en el segmento actual
        if any(arc in character_arcs for arc in segment):
            # Añadir el nodo al grafo con su atributo 'word_count'
            G_segment.add_node(person, word_count=word_count)

            # Añadir relaciones con menciones mutuas
            for mentioned in mutual_links:
                if mentioned in data:  # Asegurarse de que la mención exista en los datos
                    G_segment.add_edge(person, mentioned)

    # Guardar el grafo como archivo .gexf
    relative_path = r'./../networks/one_piece_cumulative'
    filename = f"cumulative_step_{i + 1}.gexf"
    nx.write_gexf(G_segment, os.path.join(relative_path,filename))
    print(f"Archivo guardado: {filename}")


Archivo guardado: cumulative_step_1.gexf
Archivo guardado: cumulative_step_2.gexf
Archivo guardado: cumulative_step_3.gexf
Archivo guardado: cumulative_step_4.gexf
Archivo guardado: cumulative_step_5.gexf
